# test_01_multiple_locals

In [ ]:
#|default_exp test_01_multiple_locals
#|export_as_func true

In [ ]:
#|hide
import nblite; from nblite import show_doc; nblite.nbl_export()
import tests as this_module

In [ ]:
#|top_export
import subprocess
from pathlib import Path
import shutil
import toml
import pytest
import asyncio

from repoyard import const
from repoyard.cmds import *
from repoyard._models import get_repoyard_meta, RepoPart
from repoyard._utils.sync_helper import SyncUnsafe

from tests.utils import *

In [ ]:
#|top_export
def test_01_multiple_locals():
    asyncio.run(_test_01_multiple_locals())

In [ ]:
#|set_func_signature
async def _test_01_multiple_locals(): ...

Parameters

In [ ]:
#|export
num_repos = 5

Initialise two repoyards to simulate syncing conflicts

In [ ]:
#|export
(
    sl_name, sl_rclone_path,
    [(config1, config_path1, data_path1), (config2, config_path2, data_path2)]
) = create_repoyards(num_repoyards=2)

Create some repos on repoyard 1 and sync them

In [ ]:
#|export
repo_index_names = []
async def _task(i):
    repo_index_name = new_repo(config_path=config_path1, repo_name=f"test_repo_{i}", storage_location=sl_name)
    await sync_repo(config_path=config_path1, repo_index_name=repo_index_name)
    repo_index_names.append(repo_index_name)

await asyncio.gather(*[_task(i)for i in range(num_repos)]);

Sync repometas into repoyard 2

In [ ]:
#|export
await sync_missing_repometas(
    config_path=config_path2
)

repoyard_meta2 = get_repoyard_meta(config2)
assert len(repoyard_meta2.repo_metas) == num_repos

Ensure that the repometa sync only synced the repometas, and that sync records exists

In [ ]:
async def _task(repo_meta):
    assert repo_meta.get_local_sync_record_path(config2, RepoPart.META).exists()
    assert not repo_meta.check_included(config2)

await asyncio.gather(*[_task(repo_meta) for repo_meta in repoyard_meta2.repo_metas]);

Include them into repoyard 2

In [ ]:
#|export
async def _task(repo_meta):
    await include_repo(
        config_path=config_path2,
        repo_index_name=repo_meta.index_name,
    )

await asyncio.gather(*[_task(repo_meta) for repo_meta in repoyard_meta2.repo_metas]);

Modify repos in repoyard 2 and sync

In [ ]:
#|export
async def _task(repo_meta):
    (repo_meta.get_local_part_path(config2, RepoPart.DATA) / "hello.txt").write_text("Hello, world!")
    await sync_repo(
        config_path=config_path2,
        repo_index_name=repo_meta.index_name,
    )

await asyncio.gather(*[_task(repo_meta) for repo_meta in repoyard_meta2.repo_metas]);

Sync into repoyard 1

In [ ]:
#|export
repoyard_meta1 = get_repoyard_meta(config1)

await asyncio.gather(*[
    sync_repo(
        config_path=config_path1,
        repo_index_name=repo_meta.index_name,
    )
    for repo_meta in repoyard_meta1.repo_metas
]);

Verify that the sync worked

In [ ]:
for repo_meta in repoyard_meta1.repo_metas:
    assert (repo_meta.get_local_part_path(config1, RepoPart.DATA) / "hello.txt").exists()

Create a conflict and test that it raises an exception

In [ ]:
#|export
async def _task(repo_meta):
    (repo_meta.get_local_part_path(config1, RepoPart.DATA) / "goodbye.txt").write_text("Goodbye, world!")
    await sync_repo(
        config_path=config_path1,
        repo_index_name=repo_meta.index_name,
    )
    
    with pytest.raises(SyncUnsafe):
        (repo_meta.get_local_part_path(config2, RepoPart.DATA) / "goodbye.txt").write_text("I'm sorry, world!")
        await sync_repo(
            config_path=config_path2,
            repo_index_name=repo_meta.index_name,
        )

await asyncio.gather(*[_task(repo_meta) for repo_meta in repoyard_meta1.repo_metas]);